In [1]:
from app.service.label_service import LabelManagementService

service = LabelManagementService()

In [ ]:
from app.service.text_service import UrlAnalysisService

url_analysis_service = UrlAnalysisService()
await url_analysis_service.process_url()

In [1]:
from app.infrastructure.external.cloudflare_ai_service import CloudflareAIService

text = "Hello, world!"
llm_service = CloudflareAIService()
#await cloudflare_ai_service.get_embedding(text)




In [ ]:
# 测试函数
test_text = """
年初靠着“TikTok难民风”，火起来的不只是小红书，还有多邻国。1月14日多邻国在官博上披露，普通话学习用户数同比飙增216%。

作为一款语言学习平台，按道理说内容的竞争壁垒是不够高的，尤其是多邻国的课程还是适用于通用化的场景需求。因此核心的用户来源，主要是有外语交流需求的打工人和游客，以及纯靠兴趣的学生群体。

但就算只是靠着这样的一群非刚需用户，多邻国仍然干到了大部分国家地区的教育app下载榜一。

不过，之所以这个时候开始关注多邻国，不仅仅是在于年初的事件。核心在于，海豚君认为多邻国的商业模式是一个明显优于流媒体的商业模式。它其实是游戏化、程序化的教学特征，实际符合由代码驱动的成本模式，边际投入成本比较低，而并非像流媒体一样靠持续且高昂的内容投入驱动。只要用户规模增长并达到一定规模，就能成为一个比流媒体更健康的现金奶牛。

近期一个月以来的美股回调时期，海豚君也在重点梳理一些商业模式优秀、竞争相对优势的公司，等待他们落入合理估值甚至悲观预期估值的范畴，否则价格太贵总是影响持有体验。


"""
prompt = "請以繁體中文分析本段文字，用100字左右撰寫內容摘要，捕捉核心觀點，並取一個濃縮文字重點的標題，返回JSON格式，包含2個鍵：summary和title"
result = await llm_service.analyze_text(test_text, prompt, json_response=True)

In [2]:
from app.service.text_service import TextAnalysisService
from app.service.image_service import ImageAnalysisService
from app.service.file_service import FileAnalysisService
from app.service.url_services import UrlAnalysisService
from app.service.label_service import LabelApplicationService, LabelManagementService

from bson import ObjectId

service = TextAnalysisService()
service = UrlAnalysisService()
# service = FileAnalysisService()
# service = ImageAnalysisService()

#await service.process_text()
await service.process_urls()

AttributeError: 'UrlAnalysisService' object has no attribute 'process_urls'

In [ ]:
await service.process_files()

In [ ]:
await LabelManagementService().create_label(ObjectId("67e3c391ee7d33ce57110243"), "科技股")

In [ ]:
await service.process_text()

In [ ]:
await service.process_files()

In [ ]:
from bson import ObjectId
texts = await service.get_file_texts(ObjectId("67f8cafcb9f9f148d4c9a874"))
texts

In [ ]:
len(texts[0])